<a href="https://colab.research.google.com/github/RManisha27/manisha/blob/main/Comment_Toxicity_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok tensorflow nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
import nltk
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input

In [ ]:
from google.colab import files
files.upload()

Saving train.csv to train (2).csv
Buffered data was truncated after reaching the output size limit.

In [ ]:
import re

data = []

with open("train.csv", "r", encoding="utf-8", errors="ignore") as f:
    next(f)  # skip header
    for line in f:
        match = re.match(r'^([^,]+),"(.+)",([01]),', line)
        if match:
            comment = match.group(2)
            toxic = int(match.group(3))
            data.append((comment, toxic))

df = pd.DataFrame(data, columns=["comment_text", "toxic"])
df.head()

In [ ]:
df.info()
df['toxic'].value_counts()

In [ ]:
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z]", " ", text)
    words = text.split()
    words = [stemmer.stem(w) for w in words if w not in stop_words]
    return " ".join(words)

df["clean_text"] = df["comment_text"].apply(preprocess_text)

In [ ]:
MAX_WORDS = 10000
MAX_LEN = 150

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])

X = tokenizer.texts_to_sequences(df["clean_text"])
X = pad_sequences(X, maxlen=MAX_LEN)

y = df["toxic"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
model = Sequential([
    Input(shape=(MAX_LEN,)),
    Embedding(MAX_WORDS, 128),
    LSTM(128),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_test, y_test)
)

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

In [ ]:
model.save("toxicity_model.keras")

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

!ls

In [ ]:
!ngrok config add-authtoken 371IRqoqbipmPzmoeJrsebuua3H_25nUEkEq2HdVz6imBSXuJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load model & tokenizer
model = tf.keras.models.load_model("toxicity_model.keras")

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

MAX_LEN = 150

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return text

st.title("🛡️ Comment Toxicity Detection")

user_input = st.text_area("Enter a comment")

if st.button("Analyze"):
    if user_input.strip() == "":
        st.warning("Please enter text")
    else:
        clean = preprocess_text(user_input)
        seq = tokenizer.texts_to_sequences([clean])
        pad = pad_sequences(seq, maxlen=MAX_LEN)
        pred = model.predict(pad)[0][0]

        if pred > 0.5:
            st.error(f"Toxic Comment 🚨 (Score: {pred:.2f})")
        else:
            st.success(f"Non-Toxic Comment ✅ (Score: {1-pred:.2f})")

Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null &

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("Streamlit App URL:", public_url)

Streamlit App URL: NgrokTunnel: "https://nonroyal-maura-magisterial.ngrok-free.dev" -> "http://localhost:8501"
